In [1]:
import numpy as np 
import pandas as pd 
import os

   Rather focusing on EDA I will give priority to various model    
    > Logistic regression 
    > SVM
    > XGboost
    > Naive Bayes
    > Random forest with Gridsearch
    
    

In [2]:
import pandas as pd
data_path = os.path.join('data/adult income1.csv')
df = pd.read_csv(data_path)
df.sample(5)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,native.country,income
18725,40,Private,200574,Assoc-acdm,12,Married-civ-spouse,Tech-support,Husband,White,Male,United-States,<=50K
28412,57,Self-emp-not-inc,437281,Some-college,10,Married-civ-spouse,Tech-support,Husband,White,Male,United-States,>50K
4411,23,Private,199336,Assoc-voc,11,Never-married,Craft-repair,Unmarried,White,Male,United-States,<=50K
30873,66,Private,166461,11th,7,Married-civ-spouse,Sales,Husband,White,Male,United-States,<=50K
10195,30,Private,310889,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,>50K


In [3]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'native.country', 'income'],
      dtype='object')

In [4]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
native.country    0
income            0
dtype: int64

Wow Looks Clean!            

But wait something not right in there, Yes "?" it is

In [5]:
# Count the number of rows where 'workclass' column is equal to '?'
question_mark_count = (df['workclass'] == '?').sum()

# Print the count
print("Number of rows with '?' in 'workclass' column:", question_mark_count)


Number of rows with '?' in 'workclass' column: 1778


In [6]:
# Count the number of rows where 'workclass' column is equal to '?'
question_mark_count = (df == '?').sum()

# Print the count
print("Number of rows with '?' in 'workclass' column:", question_mark_count)


Number of rows with '?' in 'workclass' column: age                  0
workclass         1778
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1785
relationship         0
race                 0
sex                  0
native.country      25
income               0
dtype: int64


In [7]:
# Count the number of rows where 'workclass' column is equal to '?'
question_mark_count = ((df['workclass'] == '?') & (df['occupation'] == '?')& (df['native.country'] == '?')).sum()

# Print the count
print("Number of rows with '?' in 'workclass' column:", question_mark_count)


Number of rows with '?' in 'workclass' column: 25


So it means that the column "occupation" has the most rows with "?" and those rows may have other columns "?" or not

In [8]:
df1=df[df['occupation']!='?']
df1.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,native.country,income
0,17,Private,148522,11th,7,Never-married,occupation,Own-child,White,Male,United-States,<=50K
1,17,Private,93235,12th,8,Never-married,occupation,Own-child,White,Female,United-States,<=50K
2,17,Private,184924,9th,5,Never-married,occupation,Own-child,White,Male,United-States,<=50K
3,17,Private,116626,11th,7,Never-married,occupation,Own-child,White,Male,United-States,<=50K
4,17,Private,209949,11th,7,Never-married,occupation,Own-child,White,Female,United-States,<=50K


In [9]:
df.shape,df1.shape

((31947, 12), (30162, 12))

Now we can change column "income" to binary values like income is 1 when ">50k" 

In [10]:
df2=df1.copy()
df2['income'] = df1['income'].apply(lambda x: 1 if x == '>50K' else 0)
df2.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,native.country,income
0,17,Private,148522,11th,7,Never-married,occupation,Own-child,White,Male,United-States,0
1,17,Private,93235,12th,8,Never-married,occupation,Own-child,White,Female,United-States,0
2,17,Private,184924,9th,5,Never-married,occupation,Own-child,White,Male,United-States,0
3,17,Private,116626,11th,7,Never-married,occupation,Own-child,White,Male,United-States,0
4,17,Private,209949,11th,7,Never-married,occupation,Own-child,White,Female,United-States,0


Now for other categorical values we have to use dummies as usual

In [11]:
categorical_cols = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
df3=df2.copy()
df3 = pd.get_dummies(df2, columns=categorical_cols, drop_first=True)
df3.head()

,age,fnlwgt,education.num,income,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
0,17,148522,7,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,17,93235,8,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,17,184924,5,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,17,116626,7,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,17,209949,7,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Lets start predicting session---

In [12]:
X = df3.drop(columns=['income'])
y = df3.income

In [13]:
X.shape,y.shape

((30162, 94), (30162,))

In [14]:
from sklearn.model_selection import train_test_split,GridSearchCV

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
X_train.shape, X_test.shape

((24129, 94), (6033, 94))

# 1. Logistic regression

In [17]:
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression(random_state=42)
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search_lr = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='accuracy')

In [18]:
best_logistic_regression = grid_search_lr.best_estimator_
y_pred_lr = best_logistic_regression.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred_lr)
print("Linear regression Accuracy:", accuracy)

Linear regression Accuracy: 0.7473893585280955


Not a good accuracy

# 2. SVC

In [19]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)
svm_classifier.fit(X_train, y_train)
y_pred_svm = svm_classifier.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVC's Accuracy:", accuracy_svm)

SVC's Accuracy: 0.7473893585280955


# 3.XGboost

In [20]:
import xgboost as xgb

xgb_classifier = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_classifier.fit(X_train, y_train)
y_pred_xgb = xgb_classifier.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGboost's Accuracy:", accuracy_xgb)

XGboost's Accuracy: 0.8319244157135753


# 4.Naive bayes

In [21]:
from sklearn.naive_bayes import GaussianNB
naive_bayes_classifier = GaussianNB()

naive_bayes_classifier.fit(X_train, y_train)

y_pred_naive_bayes = naive_bayes_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_naive_bayes)
print("Naive Bayes's Accuracy:", accuracy)

Naive Bayes's Accuracy: 0.7546825791480192


# 5.Random Forest 

In [22]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}


In [23]:
grid_search_rf = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [24]:
best_RF_model = grid_search_rf.best_estimator_
y_pred_rf=best_RF_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_rf)

accuracy

0.8339134758826454

So far Random forest and XGboost have worked better.